In [ ]:
import requests

import pandas as pd
import sqlite3

from tqdm import tqdm

In [ ]:
reqRegiones = requests.get("https://api-colombia.com/api/v1/Region").json()


In [ ]:
regiones = [{k: v for k, v in ele.items() if k != "departments"} for ele in reqRegiones]
dfRegiones = pd.DataFrame(regiones)

In [ ]:
reqDepartaments = [requests.get(f"https://api-colombia.com/api/v1/Region/{region}/departments").json() for region in tqdm(range(1, 7))]

In [ ]:
dfsDeparments = []

for region in tqdm(reqDepartaments):
    deparment = [{k: v for k, v in ele.items() if v!= None and not isinstance(v, dict)} for ele in region]
    dfsDeparments.append(pd.DataFrame(deparment))

In [ ]:
dfDeparments = pd.concat(dfsDeparments)
total_Region_Dep = dfDeparments['regionId'].nunique()
if total_Region_Dep != 6:
    raise Exception("The total of regions are 6")

In [ ]:
conn = sqlite3.connect('applications.db')
cursor = conn.cursor()
cursor.execute('DROP TABLE IF EXISTS Departmnets')
cursor.execute('DROP TABLE IF EXISTS Regions')
dfDeparments.to_sql('Departmnets', conn, if_exists='replace', index=False)
dfRegiones.to_sql('Regions', conn, if_exists='replace', index=False)